  # BioGPS (Human Cell Line)

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://biogps.org/downloads/

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter code_eval

{% set dataset = ChoiceField(
    name='dataset',
    label='Dataset',
    choices={
        'Human U133A/GNF1H Gene Atlas': "'U133A'",
        'Human NCI60 Cell Lines': "'NCI60'"
    },
    default='Human U133A/GNF1H Gene Atlas',
    section='data'
) %}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
%%appyter code_exec

output_name = 'bioGPS-' + {{dataset}}

path = 'Output/BioGPS-' + {{dataset}}
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the BioGPS Datasets',
) %}}

# Load Data

In [ ]:
%%appyter code_exec

matrix = pd.read_csv({{FileField(
    constraint='.*\.zip$',
    name='matrix', 
    label='Dataset File (zip)', 
    default='Input/BioGPS/gnf1h-gcrma.zip',
    section='data')
}}, sep=',', index_col=0)

In [ ]:
matrix.head()

In [ ]:
matrix.shape

## Load Probe Annotationsa

In [ ]:
%%appyter code_exec

gene_meta = pd.read_csv({{FileField(
    constraint='.*\.csv$',
    name='gene_meta', 
    label='Probe Annotations (csv)', 
    default='Input/BioGPS/HG-U133A.na36.annot.csv',
    section='data')
}}, sep=',', skiprows=25, 
    usecols=['Probe Set ID', 'Gene Symbol'], 
    index_col=0)

In [ ]:
gene_meta.head()

In [ ]:
gene_meta.shape

# Pre-process Data

## Map Gene to Probe

In [ ]:
matrix.index = gene_meta.reindex(matrix.index)['Gene Symbol']
matrix.index.name = 'Gene Symbol'
matrix.columns.name = 'Cell Lines'
matrix.head()

## Revert Duplicate Column Names

In [ ]:
matrix.columns = matrix.columns.map(lambda x: x.split('.')[0])
matrix.head()

## Save Unfiltered Matrix to file

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_unfiltered',
            compression='gzip', dtype=np.float32)

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
matrix = uf.map_symbols(matrix, symbol_lookup)
matrix.shape

## Merge Duplicate Genes By Rows and Duplicate Columns

In [ ]:
matrix = uf.merge(matrix, 'row')
matrix = uf.merge(matrix, 'column')
matrix.shape

## Remove Data that is More Than 95% Missing and Impute Missing Data

In [ ]:
matrix = uf.remove_impute(matrix)
matrix.head()

In [ ]:
matrix.shape

## Log2 Transform

In [ ]:
matrix = uf.log2(matrix)
matrix.head()

## Normalize Matrix (Quantile Normalize the Matrix by Column)

In [ ]:
matrix = uf.quantile_normalize(matrix)
matrix.head()

## Normalize Matrix (Z-Score the Rows)

In [ ]:
matrix = uf.zscore(matrix)
matrix.head()

## Histogram of First Sample

In [ ]:
matrix.iloc[:, 0].hist(bins=100)

## Histogram of First Gene

In [ ]:
matrix.iloc[0, :].hist(bins=100)

## Save Filtered Matrix

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_filtered', 
            ext='tsv', compression='gzip')

# Analyze Data

## Create Gene List

In [ ]:
gene_list = uf.gene_list(matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create matrix of Standardized values (values between -1, and 1)

In [ ]:
standard_matrix = uf.standardized_matrix(matrix)
standard_matrix.head()

In [ ]:
uf.save_data(standard_matrix, path, output_name + '_standard_matrix',
            ext='tsv', compression='gzip')

## Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [ ]:
plt.plot(matrix[matrix.columns[0]],
         standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

## Create Ternary Matrix

In [ ]:
ternary_matrix = uf.ternary_matrix(standard_matrix)
ternary_matrix.head()

In [ ]:
uf.save_data(ternary_matrix, path, output_name + '_ternary_matrix',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'down', path, output_name + '_gene_down_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'down', path, 
                             output_name + '_attribute_down_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(standard_matrix.T, 'cosine')
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(standard_matrix, 'cosine')
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(standard_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)